## Определение
Выпуклой оболочкой $CH(P)$ множества точек $P$ называется пересечение выпуклых множеств, содержащих все точки из $P$.


## Алгоритм Джарвиса ("заворачивания подарка")


### Алгоритм
1. Берем самую левую точку $p_0$, если таких несколько &mdash; самую нижнюю.

2. Берем точку с минимальным полярным углом относительно $p_0$, если таких несколько &mdash; ближайшую к $p_0$.
3. Добавляем выбранную точку в оболочку, проделываем то же самое с ней и т. д.
4. Если в какой-то момент найденная на шаге 2 точка совпала с $p_0$, то останавливаемся.

Пояснение: Пусть мы  на шаге 2 хотим проверить, что $p_j$ имеет имеет меньший угол, чем $p_i$. Это эквивалентно тому, что предикат поворота $turn(p_0, p_i, p_j) < 0$.

In [13]:
from utils import *
import numpy as np

def next_point(points, p):
    # находим точку с минимальным полярным углом относительно текущей точки p
    q = p
    for r in points:
        t = turn(p, q, r)
        if r != p and (q == p or angle_less(p, q, r)):
            q = r
    return q

In [ ]:
def convex_hull(points):
    # берем крайнюю точку
    hull = [min(points)]
    for p in hull:
        q = next_point(points, p)
        if q != hull[0]:
            hull.append(q)
    return hull

In [ ]:
![Пример](images/jarvis.png)

### Корректность
+ Пусть осталась точка $P$, не лежащая внутри полученной оболочки

+ $\Rightarrow$ она лежит справа от некоторого ребра $AB$

+ $P$ меньше по повороту относительно $A$, чем $B$


+ $\Rightarrow$ мы должны были выбрать $P$, а не $B$, для построения очередного ребра, когда рассматривали $A$. Противоречие.

### Сложность
+ $O(nk)$
+ $O(n^2)$ в худшем случае

## Алгоритм Грэхема




### Описание
1. Находим самую нижнюю точку $p_0$ (если таких несколько, берем самую правую), добавляем в ответ.
2. Сортируем остальные точки по полярному углу относительно $p_0$ (при равенстве углов &mdash; по расстоянию).
3. Добавляем в ответ $p_1$ — первую из отсортированных точек.
4. Берем очередную точку $p_{i+1}$. Пусть две последних точки в текущей оболочке &mdash; $h_{l-1}$ и $h_{l-2}$. Пока $h_{l-1}, h_{l-2}, p_{i+1}$ образуют не правый поворот, удаляем $h_i$.
5. Добавляем в оболочку $p_{i+1}$.
6. Делаем п.4 и 5, пока не закончатся точки.


### Корректность
![](images/graham_proof_new.png)

>  На каждом шаге множество добавленных точек является выпуклой оболочкой всех уже рассмотренных точек.


$\triangleright$
<div style="padding-left:40px">
    
  Докажем по индукции. Будем пользоваться следующим критерием выпуклости многоугольника: при обходе против часовой стрелки любые 3 соседние точки образуют не правый поворот.<br>  
 <b>База.</b> Для трех первых точек утверждение, очевидно, выполняется. <br>
    
 <b>Переход.</b> Пусть для $i$ выполнено, докажем, что верно и для $i + 1$ точки. <br>
    Пусть граница оболочки первых $i$ точек состоит из $l$ точек: $h_0, h_2, \ldots, h_{l-1}$. Пусть $h_{j}$ &mdash; точка с наибольшим индексом такая, что $turn(h_{j-1}, h_{j}, p_{i+1}) \geq 0$. Алгоритм удаляет все точки начиная с $h_{j+1}$ (или ничего, если $j = l-1$). <br> Докажем, что многоугольник $h_0 h_1\ldots h_{j}p_{i+1}$ выпуклый. Достаточно проверить только 3 поворота: $turn(h_{j-1}, h_{j}, p_{i+1}) \geq 0$ (по выбору $h_j$); $turn(h_{0}, h_{1}, p_{i+1}) \geq 0$, т.к. $p_{i+1}$ имеет больший полярный угол относительно $p_0$, чем остальные точки; $turn(h_{j}, p_{i+1}, h_0) = (h_0, h_{j}, p_{i+1}) \geq 0$ по этой же причине. Все остальные повороты такие же, как на предыдущей итерации. <br> Проверим, что все рассмотренные точки лежат нестрого внутри многоугольника $h_0 \ldots h_{j}p_{i+1}$: $p_{i+1}$ лежит на границе; на предыдущей итерации в оболочке лежали все $p_{k}, k \leq i$ (по предположению индукции), при этом текущая оболочка ограничивает более широкую часть плоскости, значит они лежат и в новой оболочке.
</div>

$\triangleleft$

### Сложность
+ Сортировка: $O(n \log n)$. 

+ При обходе точка добавляется / удаляется не более одного раза $\Rightarrow$ сложность этой части — $O(n)$. 

+ Суммарное время — $O(n \log n)$.

## Алгоритм Эндрю


### Описание
1. Возьмем самую левую и самую правую точки — $p_0$ и $p_n$
2. Разделим все множество точек на "верхние"и "нижние" — точки выше прямой $p_0p_n$ и ниже ее, соответственно.
3. Для "верхних"и "нижних"точек построим верхнюю и нижнюю оболочку соответственно. Строить будет Грэхемом, но представляя, что точка $p_0$ лежит в $\infty$ и $-\infty$ соответственно. Тогда достаточно отсортировать по $x$.
4. Объединим верхнюю и нижнюю оболочки.



###  Корректность
Грэхем корректен, а значит, верхняя и нижняя оболочки будут корректны. Тогда и вся оболочка
корректна.
### Асимптотика
Ровно такая же, как у Грэхема. Также можно отметить тот факт, что Эндрю в целом работает быстрее чем Грэхем, так как использует всего $O(n)$ поворотов (повороты не нужны при сортировке), в то время как Грэхем использует $O(n \log n)$ поворотов.

### Реализация

In [ ]:
from utils import *
from functools import *

# функция удаляет несколько последних точек текущей оболочки hull, с которыми очередная точка r 
# образует не левый поворот, и добавляет r 
def keep_left(hull, r):
    while len(hull) > 1 and turn(hull[-1], r, hull[-2]) != TURN_LEFT:
            hull.pop()
    if not hull or hull[-1] != r:
        hull.append(r)
    return hull

def convex_hull(points):
    points = sorted(points)
    l = reduce(keep_left, points, [])
    u = reduce(keep_left, reversed(points), [])
    return l.extend(u[i] for i in range(1, len(u) - 1)) or l

## Алгоритм Чена
+ Недостатком Грэхема является необходимость сортировки всех точек по полярному углу, что занимает достаточно много времени $O(n \log n)$
+ Джарвис требует перебора всех точек для каждой из $k$ точек оболочки, что в худшем случае занимает $O(n^2)$
+ Скомбинируем эти алгоритмы &mdash; получим асимптотику $O(n \log k)$


### Описание
Допустим, что $k$ нам известно.
1. Разбиваем все множество на произвольные группы по $k$ штук в каждой. Всего групп $r = n / k$.
2. Для каждой группы запускаем Грэхема.

Время — $O(r k \log k) = O(n \log k)$

3. Берем лексикографически минимальную точку. Начиная с нее ищем выпуклую оболочку Джарвисом, но перебираем не все точки, а по одной из каждой группы. 
    + На каждом шаге находим правую касательную к оболочке текущей группы и берем точку касания

Время &mdash; $O(k r \log k) = O(n \log k)$ 

Итоговое время &mdash; $O(n \log k)$

### Нахождение $k$
+ Положим $m = 2$
+ Джарвис не завершился за $m$ шагов $ \Rightarrow m < k \Rightarrow m' = m^2$ 
+ Общее время алгоритма &mdash; $\sum_{t=0}^{\lfloor{\log\log k}\rfloor} O\left(n \log(2^{2^t})\right) = O(n) \sum_{t=0}^{\lfloor{\log\log k}\rfloor} 2^t = O\left(n \cdot 2^{1+\lfloor{\log\log k}\rfloor}\right) = O(n \log k)$.

### Сложность
На втором шаге алгоритма в каждой группе оболочка ищется за $O(m \log m)$, . На третьем шаге поиск касательной занимает $O(\log m)$. Тогда поиск по всем группам займет $O(r \log m) = O(\frac{n}{m} \log m)$. Всего таких шагов будет $k$, значит общее время - $O(\frac{kn}{m} \log m)$. Итоговое время - $O(n (1 + \frac{k}{m}) \log m)$. Несложно видеть, что минимум достигается при $m = k$. В таком случае сложность равна $O(n \log k)$.

![Пример](images/intersecting_chan.svg)


### Нахождение правой касательной к выпуклому многоугольнику
Заметим, что можно за $O(1)$ проверить, что прямая $qp_i$ является правой касательной к многоугольнику $p_0p_1\ldots p_{n-1}$: для нее должно выполняться $turn(q, p_i, p_{(i+1) \bmod n}) = turn(q, p_i, p_{(i-1) \bmod n}) = TURN\_LEFT $.
На каждой итерации алгоритма будем рассматривать вершины исходного многоугольника начиная с $p_0$ с шагом $step_i$ (получившийся многоугольник обозначим $Q_i$) и строить к нему касательную. Возьмем $step_0 = n / 4$ и будем уменьшать шаг в 2 раза, пока очередной $Q_i$ не совпадет с исходным многоугольником. На нулевой итерации получим $О(1)$ точек, значит можно просто их перебрать. Пусть на $i$-ом шаге построена касательная $qp_j$. Посмотрим, какие вершины многоугольника $Q_{i+1}$ могут оказаться не с той же стороны от этой касательной, что многоугольник $q_i$. Это могут быть только точки, отстоящие от $p_j$ на $step_{i+1}$. Поэтому достаточно проверить, какая из них будет новой касательной.

![](images/r_tangent.svg)

In [1]:
from andrew import andrew_hull
from utils import *


# строим правую касательную к оболочке hull, проходящую через p,
# т.е. такую прямую, что оболочка лежит не правее ее
# возвращаем индекс точки в оболочке, через которую проходит прямая
def r_tangent(hull, p):
    length = len(hull)
    lo, hi = 0, length - 1
    if length == 1:
        return 0
    lo_prev = turn(p, hull[0], hull[-1])
    lo_next = turn(p, hull[0], hull[1])
    while lo < hi:
        c = (lo + hi) // 2
        c_prev = turn(p, hull[c], hull[(c - 1) % length])
        c_next = turn(p, hull[c], hull[(c + 1) % length])
        c_side = turn(p, hull[lo], hull[c])
        if c_prev != TURN_RIGHT and c_next != TURN_RIGHT:
            return c
        elif c_side == TURN_LEFT and (lo_next == TURN_RIGHT or
                                              lo_prev == lo_next) or \
                                c_side == TURN_RIGHT and c_prev == TURN_RIGHT:
            hi = c
        else:
            lo = c + 1
            lo_prev = -c_next
            lo_next = turn(p, hull[lo], hull[(lo + 1) % length])
    return lo


# находим лексикографически минимальную точку
# функция возвращает пару из индекса оболочки, в которой лежит точка, и индекса точки в нем
def min_hull_pt_pair(hulls):
    hull_index, point_index = 0, 0
    for i in range(len(hulls)):
        j = min(range(len(hulls[i])), key=lambda j: hulls[i][j])
        if hulls[i][j] < hulls[hull_index][point_index]:
            hull_index, point_index = i, j
    return hull_index, point_index


# выполняем одну итерацию Джарвиса, т.е. берем точку с наименьшим углом относительно последней найденной
def next_hull_pt_pair(hulls, last_indices):
    last = hulls[last_indices[0]][last_indices[1]]
    cur_min_indices = (last_indices[0], (last_indices[1] + 1) % len(hulls[last_indices[0]]))
    # перебираем все оболочки кроме той, в которой лежит последняя точка
    for hull_index in (i for i in range(len(hulls)) if i != last_indices[0]):
        # находим точку min_in_hull такую, что
        # прямая, проходящая через last и min_in_hull, лежит не левее hull
        # такая точка найдется, потому что last -- точка искомой оболочки,
        # а значит лежит не внутри hull
        min_in_hull_index = r_tangent(hulls[hull_index], last)
        cur_min, min_in_hull = hulls[cur_min_indices[0]][cur_min_indices[1]], hulls[hull_index][min_in_hull_index]
        # если точка имеет меньший полярный угол, чем текущий минимум,
        # или такой же, но удалена от последней точки на большее расстояние, обновляем минимум
        t = turn(last, cur_min, min_in_hull)
        if t == TURN_RIGHT or t == TURN_NONE and dist(last, min_in_hull) > dist(last, cur_min):
            cur_min_indices = (hull_index, min_in_hull_index)
    return cur_min_indices


def chan_hull(pts):
    for m in (1 << (1 << t) for t in range(len(pts))):
        # строим оболочки для каждого подмножества
        hulls = [andrew_hull(pts[i:i + m]) for i in range(0, len(pts), m)]
        # список пар вида (индекс оболочки, индекс точки в ней)
        # инициализируем наименьшей точкой
        hull = [min_hull_pt_pair(hulls)]
        for throw_away in range(m):
            p = next_hull_pt_pair(hulls, hull[-1])
            if p == hull[0]:
                return [hulls[h][i] for h, i in hull]
            hull.append(p)


## QuickHull


### Описание
1. Найдем самую левую точку $p_0$ и самую правую точку $p_1$ (Если таких несколько, выберем среди таких нижнюю и верхнюю соответственно).
2. Возьмем все точки выше прямой $p_0 p_1$.
3. Найдем среди этого множества точку $p_i$, наиболее отдаленную от прямой (если таких несколько, взять самую правую).
4. Рекурсивно повторить шаги 2-3 для прямых $p_0 p_i$ и $p_i p_1$, пока есть точки.
5. Добавить в ответ точки $p_0 \dots p_i \dots p_1$, полученные в п. 3.
6. Повторить пункты 2-5 для $p_1 p_0$ (то есть для "нижней" половины).
7. Ответ &mdash; объединение списков из п. 5 для верхней и нижней половины.




![Пример](images/quick_hull.png)

### Корректность

Выпуклая оболочка всего множества является объединением выпуклых оболочек для верхнего и нижнего множества. Докажем, что алгоритм верно строит оболочку для верхнего множества, для нижнего рассуждения аналогичны. Точки $p_0$ и $p_1$ принадлежат оболочке. 


- Пусть какая-то точка входит в нашу оболочку, но не должна. 
Назовем эту точку $t$. По алгоритму эта точка появилась как самая удаленная от некой прямой $t_1 t_2$. Так как $t$ не входит в оболочку, то существует прямая $t_3 t_4$ из настоящей выпуклой оболочки, что $t$ лежит снизу от прямой. Тогда какая-то из $t_3$ и $t_4$ удалена от прямой дальше $t$, что противоречит алгоритму.


- Наоборот, пусть какой-то точки $t$ в нашей оболочке нет, а должна быть.
Пойдем вниз рекурсии в те ветки, где есть $t$. В какой-то момент $t$ окажется внутри некоторого треугольника. Но тогда возникает противоречие с тем, что $t$ принадлежит выпуклой оболочке.
Таким образом, наша оболочка совпадает с истинной.

### Сложность
Пусть время, необходимое для нахождения оболочки над некой прямой и множеством точек $S$ есть $T(S)$ Тогда $T(S) = O(\left\vert{S}\right\vert) + T(A) + T(B)$, где $A, B \subset S$ &mdash; множества над полученными прямыми. Отсюда видно, что в худшем случае, алгоритм тратит $O(n^2)$. На рандомных же данных это число равно $O(n \log n)$.

In [5]:
from utils import turn, TURN_RIGHT, gen, visualize

def get_hull_points(list_pts):
    min_pt, max_pt = get_min_max_x(list_pts)
    hull_pts = quick_hull(list_pts, min_pt, max_pt)
    hull_pts = hull_pts + quick_hull(list_pts, max_pt, min_pt)
    return hull_pts


def quick_hull(points, min_pt, max_pt):
    left_of_line_pts = get_points_left_of_line(min_pt, max_pt, points)
    pt_c = point_max_from_line(min_pt, max_pt, left_of_line_pts)
    if len(pt_c) < 1:
        return [max_pt]

    hull_pts = quick_hull(left_of_line_pts, min_pt, pt_c)
    hull_pts = hull_pts + quick_hull(left_of_line_pts, pt_c, max_pt)

    return hull_pts


def get_points_left_of_line(start, end, points):
    pts = []
    for pt in points:
        if turn(start, end, pt) == TURN_RIGHT:
            pts.append(pt)

    return pts


def point_max_from_line(start, end, points):
    max_dist = 0

    max_point = []

    for point in points:
        if point != start and point != end:
            dist = distance(start, end, point)
            if dist > max_dist:
                max_dist = dist
                max_point = point

    return max_point


def get_min_max_x(list_pts):
    min_x = float('inf')
    max_x = 0
    min_y = 0
    max_y = 0

    for x, y in list_pts:
        if x < min_x:
            min_x = x
            min_y = y
        if x > max_x:
            max_x = x
            max_y = y

    return [min_x, min_y], [max_x, max_y]


# расстояние от pt до прямой, проходящей через start и end
# расстояние пропорционально площади треугольника построенного на  start, end, pt 
# (так как является его высотой), а площадь равна модулю поворота / 2
def distance(start, end, pt):
    return abs(turn_value(start, end, pt))

[[6, 9], [4, 5], [6, 3], [10, 3], [10, 9], [8, 4]]


## Оболочка простого многоугольника


### Описание
Сделаем обход Грэхема по многоугольнику, начиная с самой левой точки. Это займет $O(n)$ времени.


### Корректность

Не всегда верно, что стек после $k$-ой итерации представляет собой список вершин
корректной выпуклой оболочки для $k − 1$ вершин &mdash; очередная вершина многоугольника может повернуть сильно назад, и после исключения неправильно повернутых вершин оболочка может перестать их
заключать.

![](images/polygon_hull_bad.svg)

> Алгоритм строит корректную выпуклую оболочку.

$\triangleright$
<div style="padding-left:40px">
Построенная на $k$-ом шаге оболочка является корректной выпуклой оболочкой для всех вершин, лежащих справа от ребра $p_0p_k$. Так как в многоугольнике нет
самопересечений, это ребро пройдет через любую точку нечетное количество раз, а значит, в конечном
итоге каждая точка войдет в выпуклую оболочку.
</div>
$\triangleleft$

# Оболочка простой полилинии


![Пример](images/polyline.png)

## Алгоритм Мелькмана

Заведем дек, в котором будут поддерживаться 2 инварианта:
1. Вершины в деке – это вершины корректной выпуклой оболочки для всех рассмотренных вершин.
2. Последняя рассмотренная вершина лежит спереди и сзади дека (если она принадлежит текущей
оболочке).

При добавлении $p_9$ возможны случаи:
![Пример](images/polyline_in.svg)
+ Ничего не делаем

![Пример](images/polyline_del_begin.svg)
+ Удаляем из начала дека

![Пример](images/polyline_del_end.svg)
+ Удаляем с конца

![Пример](images/polyline_del_both.svg)
+ Удаляем с обоих концов

Как изменяется дек?
1. Если добавляемая точка оказалась в позиции 4 (внутри текущей оболочки), то ничего делать не надо. 
2. Если в позициях 1 или 3 – удаляем вершины с соответствующей стороны дека, пока не получится
корректный поворот.
3. В случае 2 удаляем по 1 вершине с обоих сторон дека.



+ В любом из случаев, в конце очередного шага необходимо положить эту вершину с обоих концов дека.
+ Заметим, что добавляемая точка не может "спрятаться" от текущей головной вершины позади всей оболочки
из-за несамопересекаемости полилинии.

Важно, что полилиния простая
![Пример](images/polyline_no.png)

### Асимптотика
+ Каждая вершина обрабатывается не более 1 раза $\Rightarrow O(n)$ 

### Корректность
Вспомним свойства выпуклой оболочки $D$ для простой полигональной цепи $P$.
1. $D$ выпуклая
2. $P$ содержится в  $D$
3. Множество вершин, из которых состоит $D$ является подмножеством вершин полигональной цепи $P$ 

Инвариант:
> Дек $ D=(d_b,d_{b+1},....d_{t-1},d_{t})$ при добавлении очередной вершины полилинии содержит в себе корректную выпуклую оболочку уже рассмотренных вершин<br>
 


Переход от $D$ к $\widetilde{D}$
+ Выпуклость сохраняется, т.к. убираем все вершины с некорректными поворотами 

+ $D$ состояла только из вершин $P$, мы добавили вершину $p_i \in P$ и удалили некоторые вершины $\Rightarrow$ $\widetilde{D}$ состоит только из вершин $P$

Пусть найдется точка $p_j$, не лежащая в $\widetilde{D}$, значит она лежит справа от ребра, проходящего через $p_i$ (т.к. все повороты с вершинами из $D$ остались левыми). 
Прямая-продолжение этого ребра пересекает оболочку в 2 точках, в которых разные знаки поворотов, в одной из них поворот левый, но мы удалили все вершины с левым поворотом. Противоречие.

![](images/polyline_proof.svg)

$\triangleright$<br>
<div style="padding-left:40px">
    Докажем по индукции. <br>
<b>База:</b> Та часть алгоритма, которая находится перед циклом `for` гарантирует, что утверждение верно для трех вершин.<br>
<b>Переход:</b> Предположим, что утверждение верно для $k$ вершин.<br>
Для начала заметим, что все вершины, которые мы пропускаем на <b>первом шаге</b> находятся внутри уже построенной выпуклой оболочки. 
Фактически, мы пропускаем очередную вершину, если она находится справа от ориентированых векторов $\mathbf{d_b d_{b+1}}$ и $\mathbf{d_{t-1}d_t}$. Более того, так как мы рассматриваем полилинию, то вершины $d_t$ и $v$ соединены, и, поскольку наша полилиния простая, ребро $\mathbf{vd_t}$ не пересекает ту часть полилинии, которая находится между $d_{b+1}$ и $d_{t-1}$. Следовательно, $v$ должна лежать внутри полигона, состоящего из вершин лежащих в $D$. 

Если же мы не пропускаем очередную вершину, то для нее можно предположить, что верно  $turn(d_b,d_{b+1},v)=TURN\_RIGHT$ или $(d_{t-1},d_t,v)=TURN\_RIGHT$ (или оба), а значит для нее выполнятся <b>второй</b> или <b>третий шаги</b>(или оба). <br>
Теперь представим, что у нас был дек $D=(d_b,...d_t)$, и после обработки очередной вершины $v$ полилинии $P$   получился новый дек $\widetilde{D}=(d_k,\dots,d_m)$, где $d_k=d_m=v$. Давайте докажем, что вышеописанные <b>условия 1, 2</b> и <b>3</b> выполняются для $\widetilde{D}$, а значит, $\widetilde{D}$ является корректной выпуклой оболочкой.

Выполнение <b>условия 3</b> очевидно, поскольку мы строим $\widetilde{D}$ из вершин $P$.<br>
Легко понять, что уже рассмотренный кусок полилинии $P$ содержится в $\widetilde{D}$, поскольку он содержался в $D$,а на втором и третьем шаге мы удаляли из дека только в том случае, если текущая вершина дека $d_k$ образует левый поворот с векторами $\mathbf{d_{t-1} v}$ и $\mathbf{v d_{b+1}}$. Таким образом, выполнено <b>условие 2</b>.<br>
Чтобы доказать,  что $\widetilde{D}$ выпуклый, покажем, что он образует замкнутую простую кривую, такую, что для нее верно $turn(d_i,d_{i+1},d_{i+2})\neq TURN\_RIGHT$ для $i=k,\dots,m-2$, и $turn(d_{m-1},d_{m},d_{k+1})\neq TURN\_RIGHT$. Поскольку $D$ образовывал простую полилинию, то $\widetilde{D}$ может иметь самопересечения только в случае, если  $\mathbf{vd_{k+1}}$ и $\mathbf{d_{m-1}v}$ пересекают ребра той части полилинии, по которой был построен $D$, а такого не может быть, потому что сама $P$ изначально была простой. Тот же факт, что $turn(d_i,d_{i+1},d_{i+2})\neq TURN\_RIGHT$ для $i=k,\dots,m-2$, был уже изначально верен для $D$ и является результатом выполнения <b>шагов два</b> и <b>три</b>. <br>
Докажем от противного, что  $turn(d_{m-1},d_{m},d_{k+1})\neq TURN\_RIGHT$. Пусть $turn(d_{m-1},d_{m},d_{k+1})=TURN\_RIGHT$  тогда вершина $v=d_m$ должна находиться слева от ориентированного ребра $\mathbf{d_{m-1}d_{k+1}}$, и в то же время должна находиться по левую сторону от полилинии $(d_{k+1},\dots,d_{m-1})$. Значит, что $v$ находится внутри $D$, и должна была быть отброшена на <b>шаге один</b>. Таким образом, <b>условие 1</b> также выполнено.
</div>
$\triangleleft$